**Содержание**<a id='toc0_'></a>    
- [Векторная модель текста и классификация длинных текстов](#toc1_)    
  - [Разреженные векторные модели](#toc1_1_)    
  - [Закон Ципфа](#toc1_2_)    
  - [TF-IDF](#toc1_3_)    
  - [Точечная взаимная информация (pointwise mutual information, PMI)](#toc1_4_)    
- [Создаём нейросеть для работы с текстом](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Векторная модель текста и классификация длинных текстов](#toc0_)



Ве́кторная моде́ль (англ. vector space model) — в информационном поиске представление коллекции документов векторами из одного общего для всей коллекции векторного пространства.

Документ в векторной модели рассматривается как неупорядоченное множество термов. Термами в информационном поиске называют слова, из которых состоит текст, а также такие элементы текста, как, например, *2010, II-5* или *Тянь-Шань*.

Более формально

$$d_j = (w_{1j}, w_{2j}, …, w_{nj})$$
где $d_j$ — векторное представление j-го документа, $w_{ij}$ — вес i-го терма в j-м документе, n — общее количество различных термов во всех документах коллекции.

*Косинусное сходство* — это мера сходства между двумя векторами предгильбертового пространства, которая используется для измерения косинуса угла между ними.

Если даны два вектора признаков, A и B, то косинусное сходство, $cos(θ)$, может быть представлено используя скалярное произведение и норму:

$${\displaystyle {\text{similarity}}=\cos(\theta )={A\cdot B \over \|A\|\|B\|}={\frac {\sum \limits _{i=1}^{n}{A_{i}\times B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{(A_{i})^{2}}}}\times {\sqrt {\sum \limits _{i=1}^{n}{(B_{i})^{2}}}}}}}$$

В случае информационного поиска, косинусное сходство двух документов изменяется в диапазоне от 0 до 1, поскольку частота терма (веса tf-idf) не может быть отрицательной. Угол между двумя векторами частоты терма не может быть больше, чем 90°.

## <a id='toc1_1_'></a>[Разреженные векторные модели](#toc0_)
Ещё их называют "методом мешка слов". 

Они хорошо работают, когда класс документа соответствует его тематике. Как правило, тематика документа хорошо описывается составом словаря, который используется в этом документе, а также частотами слов, а не тем, как именно они употребляются в документе, не структурой фраз. 

Тогда каждый документ описывается длинным вектором размерности порядка десятков или сотен тысяч элементов. Большая часть — нули. Для каждого слова в документе мы имеем какое-то вещественное число. 

Ранее мы уже выяснили, что модели должны работать лучше, когда веса слов отличаются — чем значимее слово, тем больше его вес. 

Однако, как именно считать эти веса? 

**Простейший вариант**
- взвешивать слова по количеству их употреблений в документе.[1,2] 

Элементарно — здесь мы видим несколько наиболее частотных слов из статьи Википедии про машинное обучение. Веса слов — это просто целые числа. 

Естественно, данный подход имеет недостатки: 
- вес слова зависит от длины документа. В длинных документах слова имеют больший вес, как будто бы они более значимы, но это не так. 
- самые частотные слова — это союзы, предлоги, местоимения... Они встречаются везде, но абсолютно неинформативны и редко бывают полезны для каких-либо задач классификации. 

Преодолеть эти недостатки призвана **нормировка**:

$$nw_i = \frac{w_i}{\sqrt{\sum_i w_j^2}}$$

- отнормируем вектор документа на его длину (например по L2-норме (евклидовой)). 

Тогда веса слов будут зависеть от длины документа гораздо слабее, но они всё равно **будут зависеть**, так как с увеличением длины документа расширяется используемый словарный запас. Однако, по-прежнему, предлоги и союзы — это самые значимые слова. 

Нас это не совсем устраивает. *Хотя иногда в задачах классификации предлоги, союзы и местоимения всё-таки могут быть полезны - особенно когда важен стиль написания текста*.

[1] Количество уникальных слов в документе - закон Ципфа https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD_%D0%A5%D0%B8%D0%BF%D1%81%D0%B0  
[2] Векторизация текстов через подсчёт количества словоупотреблений https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

## <a id='toc1_2_'></a>[Закон Ципфа](#toc0_)
- один из фундаментальных эмпирических законов лингвистики (и не только лингвистики, на самом деле). 

Возьмём большую коллекцию документов, посчитаем для каждого слова в этой коллекции частоту его встречаемости, то есть количество документов, в которых это слово используется, потом отсортируем полученный список по убыванию частот и получим примерно (гиперболически убывающий) график. 

Плотность распределения Ципфа (класс степенных распределений):

$$f(rank, s, N) = \frac{1}{Z(s, N) \cdot rank^s}$$

$rank$ - исследуемая сл.вел. - порядковый номер слова после сортировки по убыванию частоты  
$s$ - коэфф-т скорости убывания вероятности  
$N$ - количество слов  
$Z(s, N)=\sum_{i=1}^N j^{(-s)}$ - нормализационная функция, чтобы распределение стало распределением.

Закон Ципфа - это эмпирическая закономерность распределения частот слов естественных языков (или достаточно длинных тектов): частотность n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n. Второе по используемости слово встречается примерно в два раза реже, чем первое, третье — в три раза реже, чем первое, и так далее.

- Christopher D. Manning, Prabhakar Raghavan and Hinrich Schütze, Introduction to Information Retrieval, Cambridge University Press. 2008. https://nlp.stanford.edu/IR-book/
- Закон Ципфа https://en.wikipedia.org/wiki/Zipf%27s_law

PS: Американский специалист по биоинформатике Вэньтянь Ли[en] предложил статистическое объяснение закона Ципфа, доказав, что случайная последовательность символов также подчиняется этому закону[12]. Автор делает вывод, что закон Ципфа, по-видимому, является чисто статистическим феноменом, который не имеет отношения к семантике текста и имеет поверхностное отношение к лингвистике. 

В общих чертах доказательство этой теории состоит в следующем. Вероятность случайного появления какого-либо слова длиной n в цепочке случайных символов уменьшается с ростом n в той же пропорции, в какой растёт при этом ранг этого слова в частотном списке (порядковой шкале). Потому произведение ранга слова на его частотность есть константа.

Этот "закон" также работает в отношении распределения городской системы: город с самым большим населением в любой стране в два раза больше, чем следующий по размеру город, и так далее.

В 1999 году экономист Ксавье Габэ описал закон Ципфа как пример степенного закона: если города будут расти случайным образом с одинаковым среднеквадратичным отклонением, то в пределе распределение будет сводиться к закону Ципфа

**Пример:**

Пусть в некотором языке есть $N=3$ слова - А, Б и В. Их ранги - 1, 2 и 3 (нумерация рангов начинается с 1).   
Найдите вероятности встретить каждое из этих слов в тексте при условии, что относительные частоты слов распределены по Ципфу с $s=2$.

In [1]:
p = [1 / (1 + 1/4 + 1/9), 1 / 4 / (1 + 1/4 + 1/9), 1 / 9 / (1 + 1/4 + 1/9)]
list(map(lambda x: round(x, 5), p))

[0.73469, 0.18367, 0.08163]

In [2]:
%load_ext rpy2.ipython 

In [3]:
%%R

v = c(1, 2, 3)^(-2)
res = 1 / c(1, 2, 3)^2 / sum(v)
res

[1] 0.73469388 0.18367347 0.08163265


Найдите количество слов, которые встречаются менее, чем в 10 из 10000 документов, если предполагать, что вероятность встретить слово в документе распределена по Ципфу с параметром $s = 2$, количество слов в словаре $N = 1000$. Ранги нумеруются с 1.

In [4]:
%%R

p <- 10 / 10000                 # пороговая вероятность
s <- 2                          # к-т убывания вероятности
N <- 1000                       # размер словаря
ranks <- (1:N) ^ s
Z <- sum((1:N) ^ (-s))
f <- 1 / Z / ranks              # вероятности распределения Ципфа
sum(f < p)

[1] 976


Выводы:

- частотных слов очень мало. Они слабо информативны, так как встречаются практически во всех документах. 
- редких слов очень много — если мы какое-нибудь редкое слово встречаем в документе, то мы с большой уверенностью можем сказать, к какой тематике он относится.

Нужно как-то свернуть эти два фактора в одну метрику.

## <a id='toc1_3_'></a>[TF-IDF](#toc0_)

TF - term frequency - значимость слова w в рамках документа d (количество употреблений слова в документе к длине документа)

$$TF(w, d) = \frac{WordCount(w,d)}{Length(d)}$$

IDF - inverse document frequency - обратная частота слова в документах, специфичность слова (размер коллекции делим на количество документов, в которых слово употребляется)

$$IDF(w, c) = \frac{Size(c)}{DocCount(w, c)}$$

Тогда итоговый вес слова в документе (TFIDF):

$$TFIDF(w, d, c) = TF(w, d) \cdot IDF(w, c)$$

На практике, TF и/или IDF часто логарифмируют, например, так: 

$TF_l(w, d) = \log(TF(w, d) + 1)$ - полезно, когда в датасете документы сильно отличаются по длине. Это позволяет сделать распределение весов слов менее контрастным и уменьшить его дисперсию.

$IDF_l(w, d) = \log(IDF(w, c) + 1)$ - полезно, когда документы сильно отличаются по лексике/набору слов (?) 

**Алгоритм:**

1. Нормализация текста (стемминг/лемматизация), выделение базовых элементов (символы, токены, n-граммы)
2. Создание частотного словаря Doccount(w, c) для всех w
3. Убрать из словаря выбросы по частоте
4. Для каждого документа d:
    - для каждого слова w из d найти WordCount(w, d)
        
        записать в результирующий вектор в позицию w значение TFIDF(w, d, c)
    - записать вектор документа в таблицу признаков документов коллекции

Особенности TFIDF:

- это способ отбора категориальных признаков
- не использует информацию о метках документов (теряем информацию, если метки есть, но преимущество, если меток нет)

## <a id='toc1_4_'></a>[Точечная взаимная информация (pointwise mutual information, PMI)](#toc0_)

Альтернативный способ взвешивания признаков [1,2,3]:

- измеряется между двумя случайными событиями или реализациями двух случайных величин. 

- характеризует, насколько сильнее мы будем ожидать первое событие, если перед этим пронаблюдаем второе[4] (по сравнению с нашими априорными ожиданиями). 

Для задачи классификации текстов:

$$pmi(l, w) = \log \frac{p(w,l)}{p(w)\cdot p(l)} = \log \frac{p(l|w)}{p(l)} = \log \frac{p(w|l)}{p(w)}$$

$l$ - коллекция документов, соответствующая метке класса $L$  
$w$ - слово из словаря  
$p(w, l) = DocCount(w, l) / Size(l)$ - вероятность встретить слово w в документе класса L  
$p(w) = \sum_l DocCount(w, l) / \sum_l Size(l)$ - маржинальная вероятность употребления слова w  
$p(l) = Size(l) / \sum_m Size(m)$ - маржинальная вероятность встретить документ класса L

- Взаимная информация минимальна, когда события w и l несовместны (если происходит одно, второе не может произойти).
- Взаимную информацию можно применить для оценки совместной встречаемости слов pmi(w_1, w_2)
- Взаимная информация максимальна, когда события w и l всегда появляются одновременно.

Все три варианта формулы эквивалентны. 

Т.о. у нас есть два события. Первое — "L": "мы наблюдаем документ из класса L". Второе событие — "W": "мы видим в документе слово W". Все вероятности вычисляются по классическому определению вероятности, то есть как отношение количества положительных исходов к общему числу исходов. 

**Взаимная информация (PMI)** — это тоже способ взвешивания и отбора категориальных признаков.[5] В первую очередь, он подходит для задач классификации. В задачах регрессии его тоже можно применять — например, дискретизировав целевое распределение, но это уже сложнее. Он требует наличия двух событий, что усложняет его применение в задачах обучения без учителя, хотя он используется для получения плотных векторных представлений слов.

[1] Точечная взаимная информация https://en.wikipedia.org/wiki/Pointwise_mutual_information  
[2] Взаимная информация - мера связанности двух случайных величин - мат.ожидание PMI https://en.wikipedia.org/wiki/Mutual_information  
[3] Применение PMI для представления смыслов слов (про это будут ещё лекции 3.2 и 3.3) Levy, Omer, and Yoav Goldberg. "Neural word embedding as implicit matrix factorization." Advances in neural information processing systems. 2014. https://papers.nips.cc/paper/5477-neural-word-embedding-as-implicit-matrix-factorization.pdf  
[4] Маргинальное (частное, маржинальное) распределение вероятностей https://en.wikipedia.org/wiki/Marginal_distribution  
[5] Ещё несколько способов взвешивания и отбора признаков https://scikit-learn.org/stable/modules/feature_selection.html  

# <a id='toc2_'></a>[Создаём нейросеть для работы с текстом](#toc0_)

**Дано:**

- коллекция документов по тематике - положительные примеры (класс 1)
- фоновая коллекция документов - отрицательные примеры (класс 0)
- длина документов - от нескольких предложений (для твиторов используются другие подходы)

**Задача:**

Построить бинарный классификатор
$$Classifier: \mathbb R^d \longrightarrow \{0, 1\}$$

$d$ - размер словаря

**Решение:**

Представление документов - **мешок слов**  
Решающее правило - **логистическая регрессия**

**Модель:**

$$\hat y(x) = \sigma (w^Tx+b)$$

$x \in \mathbb R^d$ - вектор признаков, если надо, дополненный единицей для смещения   
$\hat y \in \mathbb R, \hat y \in [0,1]$ - вероятность выпадения 1 для сл.вел. $y \in \{0,1\}$  
$w \in \mathbb R^d$ - вектор весов  
$\sigma(x) = \frac {1}{1+e^{-x}}$ - логистическая функция (сигмоида)

Это можно рассматривать как искусственный нейрон с $d$ входами и 1 выходом.

**Алгорим (процесс обучения):**

Функция потерь - бинарная кросс-энтропия

$$BCE(\hat y, y) = -y \log \hat y - (1-y) \log(1-\hat y) \longrightarrow \min$$

- первое слагаемое - штраф за ложноположительный отве, второе - за ложноотрицательный

Алгоритм настройки (поиск весов модели) - градиентный спуск.

**Дано:**

- несколько коллекций документов по разным тематикми
- длина документов - от нескольких предложений (для богомерзких твиторов используются другие подходы)

**Задача:**

Построить n-арный классификатор
$$Classifier: \mathbb R^d \longrightarrow \{1, ..., c\}$$

$d$ - размер словаря

**Решение:**

Представление документов - **мешок слов**  
Решающее правило - **логистическая регрессия**

**Модель:**

$$\hat y(x) = \text{softmax} (W \cdot x)$$

$x \in \mathbb R^d$ - вектор признаков, если надо, дополненный единицей для смещения   
$\hat y \in \mathbb R^с, \sum_{i=1}^c \hat y_i = 1$ - на выходе вектор - распределение вероятностей отнесения к классу $i$     
$W \in \mathbb R^{c \times d}$ - матрица весов (количество классов Х количество признаков)   
$\text {softmax}(x) = \left \{ \frac {e^{x_j}}{\sum_{j=1}^c e^{x_j}} \right \}_i$ - функция активации softmax: отображает вещественный вектор в такой вектор положительных чисел, что сумма координат всегда равна 1.

Это можно рассматривать как искусственный нейрон с $d$ входами и $c$ выходами.

**Алгорим (процесс обучения):**

Функция потерь - кросс-энтропия

$$CE(\hat y, y) = - \sum_{i=1}^c y_i \log \hat y_i \longrightarrow \min$$

- т.к. классы взаимоисключающие, то достаточно штрафовать за ложноотрицательные предсказания

Алгоритм настройки (поиск весов модели) - градиентный спуск.

Мы поговорили про логистическую регрессию (или однослойную нейронную сеть), которая, по сути, является линейной регрессией с функцией активации в виде сигмоиды или софтмакса, а также мы рассмотрели варианты для двух и нескольких классов. 

In [5]:
from math import log

def TBCE(y_pred, y_true=(1, 1)):
    r = 0
    for i in range(len(y_true)):
        r += BCE(y_pred[i], y_true[i])
    return r

def BCE(y_hat, y):
    result = -y * log(y_hat) - (1 - y) * log(1-y_hat)
    return result

print(TBCE([0.99, 0.45]), TBCE([0.99, 0.01]))
print(TBCE([0.99, 0.01]), TBCE([0.5,0.5]))
print(TBCE([0.5,0.5]), TBCE([0.99, 0.01]))
print(TBCE([0.99,0.45]), TBCE([0.65,0.65]) )
print(TBCE([0.99, 0.01]), TBCE([0.99, 0.45]))

0.8085580320712731 4.615220521841592
4.615220521841592 1.3862943611198906
1.3862943611198906 4.615220521841592
0.8085580320712731 0.8615658321849085
4.615220521841592 0.8085580320712731


Найдите в общем виде производную функции потерь $\frac{\partial BCE(\hat{y}, y)}{\partial w}$ и $\frac{\partial BCE(\hat{y}, y)}{\partial b}$ и запишите в ответ её формулу.

Ну там еще с регуляризацией производные, вот это все..

In [6]:
from sympy import * 


hy, y, x, w, b, c, t = symbols("hy y x w b c t")
hy = 1 / (1 + exp(- w*x - b))
BCE = -y*log(hy) - (1 - y) * log(1 - hy)
BCE_w = diff(BCE, w).simplify() 
print(BCE_w)

BCE_b = diff(BCE, b).simplify() 
print(BCE_b)

loss_reg = BCE + c * (w**2 + b**2)
loss_w = diff(loss_reg, w).simplify() 
print(loss_w)

# diff_str = str(BCE_w)
# expr = parsing.sympy_parser.parse_expr(diff_str)
# sample_value = expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1))
# print(sample_value)

Set(BCE_w, BCE_b, loss_reg)

x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)
(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)
(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1)


Set(x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1), (-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1), c*(b**2 + w**2) - y*log(1/(exp(-b - w*x) + 1)) - (1 - y)*log(1 - 1/(exp(-b - w*x) + 1)))

Напишите функцию для вычисления точечной взаимной информации двух случайных событий.

$$pmi(a, b) = \log \frac {p(a, b)} {p(a) p(b)}$$


In [7]:
import sys
import numpy as np


def parse_array(s):
    return np.array([int(s.strip()) for s in s.strip().split(' ')])

def read_array():
    return parse_array(sys.stdin.readline())

def calculate_pmi(a, b):
    pa = np.mean(a)
    pb = np.mean(b)
    pab = np.mean(a & b)

    return np.log(pab/pa/pb)

a = parse_array('1 0 0 1 1 0')
b = parse_array('1 0 0 0 1 0')
pmi_value = calculate_pmi(a, b)

print('{:.6f}'.format(pmi_value))

0.693147


В этом задании мы предлагаем вам собрать регулярное выражение из "деталей" так, чтобы оно выделяло в отдельные токены знаки препинания, числа и слова.

А именно:

    Числа с плавающей точкой вида 123.23 выделяются в один токен. Десятичным разделителем может быть точка или запятая.
    Число может быть отрицательным: иметь знак -123.4−123.4
    Целой части числа может вовсе не быть: последовательности  -0.15−0.15  и  -.15−.15   означают одно и то же число.
    При этом числа с нулевой дробной частью не допускаются:  строка "12345.12345." будет разделена на два токена "12345" и "."
    Идущие подряд знаки препинания выделяются каждый в отдельный токен.
    Наконец множество букв в словах ограничивается только кириллическим алфавитом (33 буквы, включая букву ё).
    Обратите внимание, что в результате токенизации не должно получаться пустых токенов.

In [8]:
import re

lines = ["Контактный телефон: 123123.", 
         "Что-нибудь надо придумать.", 
         "Значение числа Е=2.7182.", 
         "Демон123, как тебя зовут в реале?", 
         "-1-.15=-1.15", 
         "- 1 - .15 = -1.15", 
         "Какого ;%:?* тут происходит?"]

# модифицируйте это регулярное выражение
TOKENIZE_RE = re.compile(r'[а-яё]+|-?\d*[.,]?\d+|\S', re.I)


def tokenize(txt):
    return TOKENIZE_RE.findall(txt)

for line in lines:
    print(' '.join(tokenize(line.strip().lower())))

контактный телефон : 123123 .
что - нибудь надо придумать .
значение числа е = 2.7182 .
демон 123 , как тебя зовут в реале ?
-1 -.15 = -1.15
- 1 - .15 = -1.15
какого ; % : ? * тут происходит ?


Дана следующая коллекция текстов. Постройте словарь (отображение из строкового представления токенов в их номера) и вектор весов (DF). Вывести в порядке возрастания частоты встречаемости.

In [9]:
corpus = [
    'Казнить нельзя, помиловать. Нельзя наказывать.',
    'Казнить, нельзя помиловать. Нельзя освободить.',
    'Нельзя не помиловать.',
    'Обязательно освободить.']

# Получение DF
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(smooth_idf=False, use_idf=True)
vectorizer.fit_transform(corpus)

# из IDF  в DF
word_doc_freq = 1 / np.exp(vectorizer.idf_ - 1) ## ВОТ ОСОБЕННОСТЬ ПЕРЕВОДА ИЗ IDF В DF

vocabulary = vectorizer.get_feature_names_out()
vocabulary = vocabulary[np.argsort(word_doc_freq)]
word_doc_freq = word_doc_freq[np.argsort(word_doc_freq)]

print(*vocabulary)
print(*np.round(word_doc_freq, 2))

наказывать не обязательно казнить освободить нельзя помиловать
0.25 0.25 0.25 0.5 0.5 0.75 0.75


Постройте матрицу признаков для текстов с предыдущего шага с использованием словаря и вектора весов, полученного на предыдущем шаге. Используйте взвешивание $lTFIDF = \ln(TF + 1) \cdot IDF$.

Значения признаков следует отмасштабировать так, чтобы для каждого признака его среднее значение по выборке равнялось 0, а среднеквадратичное отклонение 1: $x^{scaled}_{i} = \frac{x_{i} - E(x)} {\sigma(x)}$.

В результате масштабирования для каждого столбца матрицы признаков среднее должно равняться 0, а среднеквадратичное отклонение 1.

При расчёте среднеквадратического отклонения необходимо использовать скорректированную оценку $\sigma=\sqrt{\frac{\sum_{i-1}^n(x_i - E(x))^2}{n - 1}}$. Чтобы получить такую оценку с помощью numpy, необходимо передать параметр ddof=1: 

    feature_matrix = np.zeros((num_docs, num_feats))
    feats_std = feature_matrix.std(0, ddof=1)

In [10]:
IDF = np.log(1 / word_doc_freq) + 1     # перевели обратно в IDF (специфичность слов в корпусе)

def tokenize(doc):                      # простой токенизатор
    return re.findall(r'[\w\d]+', doc.lower())

def count(word, doc):                   # простой счетчик
    tokens = tokenize(doc)
    doc_len = len(tokens)
    return tokens.count(word) / doc_len

vcount = np.vectorize(count)
TF = vcount(word=vocabulary, 
            doc=np.array(corpus).reshape(-1, 1))   # корпус развернут в столбец

lTFIDF = np.log(TF + 1) * IDF           # сглаженный как просили TFIDF                         

res = (lTFIDF - lTFIDF.mean(axis = 0)) / lTFIDF.std(0, ddof=1)      # центрированный масштабированный TFIDF

[print(*row) for row in np.round(res, 2)]
pass


1.5 -0.5 -0.5 0.87 -0.76 0.6 0.16
-0.5 -0.5 -0.5 0.87 0.18 0.6 0.16
-0.5 1.5 -0.5 -0.87 -0.76 0.29 1.04
-0.5 -0.5 1.5 -0.87 1.34 -1.48 -1.36


Семинары в директории `./stepic-dl-nlp`